In [ ]:
from parse import load_ps
import utils
import numpy as np
import math
import matplotlib.pyplot as plt

from scipy.signal import medfilt
from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go

In [ ]:
class BodyValue:
    def __init__(self,pose):
        self.pose = pose
    # 상박
    def upper_arm_vectors_r(self):
        return [self.pose.rshoulder.x - self.pose.relbow.x, self.pose.rshoulder.y - self.pose.relbow.y]
    def upper_arm_vectors_l(self):
        return [self.pose.lshoulder.x - self.pose.lelbow.x, self.pose.rshoulder.y - self.pose.relbow.y]
    # 하박
    def lower_arm_vectors_r(self): 
        return [self.pose.relbow.x - self.pose.rwrist.x, self.pose.relbow.y - self.pose.rwrist.y]

    def lower_arm_vectors_l(self):
        return [self.pose.lelbow.x - self.pose.lwrist.x, self.pose.lelbow.y - self.pose.lwrist.y]
    # 목 - 엉덩이
    def torso_vectors(self):
        return [self.pose.neck.x - (self.pose.rhip.x + self.pose.lhip.x)/2, self.pose.neck.y - (self.pose.rhip.y + self.pose.lhip.y)/2]
    # 허벅지
    def upper_leg_vectors_r(self):
        return [self.pose.rhip.x - self.pose.rknee.x, self.pose.rhip.y - self.pose.rknee.y]
    def upper_leg_vectors_l(self):
        return [self.pose.lhip.x - self.pose.lknee.x, self.pose.lhip.y - self.pose.lknee.y]
    # 종아리
    def lower_leg_vectors_r(self):
        return [self.pose.rknee.x - self.pose.rankle.x, self.pose.rknee.y - self.pose.rankle.y]
    def lower_leg_vectors_l(self):
        return [self.pose.lknee.x - self.pose.lankle.x, self.pose.lknee.y - self.pose.lankle.y]
    # 발 사이 거리
    def between_feet_vectors(self):
        return[self.pose.rankle.x - self.pose.lankle.x, self.pose.rankle.y - self.pose.lankle.y]
    # 몸 - 팔 각도
    def upper_arm_torso_angle_r(self):
        return math.degrees(np.arccos(np.clip(np.dot(self.upper_arm_vectors_r(),self.torso_vectors()),-1.0,1.0)))
    def upper_arm_torso_angle_l(self):
        return math.degrees(np.arccos(np.clip(np.dot(self.upper_arm_vectors_l(),self.torso_vectors()),-1.0,1.0)))
    # 상박 - 하박 각도
    def upper_lower_arm_angle_r(self):
        return math.degrees(np.arccos(np.clip(np.dot(self.upper_arm_vectors_r(),self.lower_arm_vectors_r()),-1.0,1.0)))
    def upper_lower_arm_angle_l(self):
        return math.degrees(np.arccos(np.clip(np.dot(self.upper_arm_vectors_l(),self.lower_arm_vectors_l()),-1.0,1.0)))
    # 허벅지 - 종아리 각도
    def upper_lower_leg_angle_r(self):
        return math.degrees(np.arccos(np.clip(np.dot(self.upper_leg_vectors_r(),self.lower_leg_vectors_r()),-1.0,1.0)))
    def upper_lower_leg_angle_l(self):
        return math.degrees(np.arccos(np.clip(np.dot(self.upper_leg_vectors_l(),self.lower_leg_vectors_l()),-1.0,1.0)))

In [ ]:

files = utils.files_in_order('poses_compressed/baseball/Accurate_baseball')

for filename in files:
    print("="*30)
    print("Starting:", filename)
    ps = load_ps("poses_compressed/baseball/Accurate_baseball/"+filename)
    poses = ps.poses
    
    # 발 사이 거리
    between_feet_vectors = []
    # 몸 - 팔 각도
    upper_arm_torso_angle_r = []
    upper_arm_torso_angle_l = []
    # 상박 - 하박 각도
    upper_lower_arm_angle_r = []
    upper_lower_arm_angle_l = []
    # 허벅지 - 종아리 각도
    upper_lower_leg_angle_r=[]
    upper_lower_leg_angle_l=[]

    for pose in poses:
        between_feet_vectors.append(((BodyValue(pose).between_feet_vectors()[0])**2 + (BodyValue(pose).between_feet_vectors()[1])**2)**1/2)

        upper_arm_torso_angle_r.append(BodyValue(pose).upper_arm_torso_angle_r())
        upper_arm_torso_angle_l.append(BodyValue(pose).upper_arm_torso_angle_l())

        upper_lower_arm_angle_r.append(BodyValue(pose).upper_lower_arm_angle_r())
        upper_lower_arm_angle_l.append(BodyValue(pose).upper_lower_arm_angle_l())

        upper_lower_leg_angle_r.append(BodyValue(pose).upper_lower_leg_angle_r())
        upper_lower_leg_angle_l.append(BodyValue(pose).upper_lower_leg_angle_l())


    between_feet_vectors_filtered = medfilt(medfilt(np.array(between_feet_vectors), 5), 5)

    upper_arm_torso_angle_r_filtered = medfilt(medfilt(np.array(upper_arm_torso_angle_r), 5), 5)
    upper_arm_torso_angle_l_filtered = medfilt(medfilt(np.array(upper_arm_torso_angle_l), 5), 5)

    upper_lower_arm_angle_r_filtered = medfilt(medfilt(np.array(upper_lower_arm_angle_r), 5), 5)
    upper_lower_arm_angle_l_filtered = medfilt(medfilt(np.array(upper_lower_arm_angle_l), 5), 5)

    upper_lower_leg_angle_r_filtered = medfilt(medfilt(np.array(upper_lower_leg_angle_r), 5), 5)
    upper_lower_leg_angle_l_filtered = medfilt(medfilt(np.array(upper_lower_leg_angle_l), 5), 5)

    fig = plt.figure(figsize=(20,20))

    print(between_feet_vectors_filtered.shape)
    print(upper_arm_torso_angle_r_filtered.shape)
    print(between_feet_vectors_filtered.shape[0])
    ax1=fig.add_subplot(4,2,1)
    ax1.scatter(np.arange(between_feet_vectors_filtered.shape[0]),between_feet_vectors_filtered, c='r',alpha=0.5)
    plt.title('Distance between feet')
    plt.xlabel('Frames')
    # plt.ylim(0,180) 
    ax2=fig.add_subplot(4,2,3)
    ax2.scatter(np.arange(upper_arm_torso_angle_r_filtered.shape[0]),upper_arm_torso_angle_r_filtered, c='r',alpha=0.5)
    plt.title('Angle between upper arm & torso(right)')
    plt.xlabel('Frames')
    # plt.ylim(0,180) 
    ax3=fig.add_subplot(4,2,4)
    ax3.scatter(np.arange(upper_arm_torso_angle_l_filtered.shape[0]),upper_arm_torso_angle_l_filtered, c='r',alpha=0.5)
    plt.title('Angle between upper arm & torso(left)')
    plt.xlabel('Frames')
    # plt.ylim(0,180) 
    ax5=fig.add_subplot(4,2,5)
    ax5.scatter(np.arange(upper_lower_arm_angle_r_filtered.shape[0]),upper_lower_arm_angle_r_filtered, c='r',alpha=0.5)
    plt.title('Angle between upper arm & lower arm(right)')
    plt.xlabel('Frames')
    # plt.ylim(0,180) 
    ax6=fig.add_subplot(4,2,6)
    ax6.scatter(np.arange(upper_lower_arm_angle_l_filtered.shape[0]),upper_lower_arm_angle_l_filtered, c='r',alpha=0.5)
    plt.title('Angle between upper arm & lower arm(left)')
    plt.xlabel('Frames')
    # plt.ylim(0,180) 
    ax7=fig.add_subplot(4,2,7)
    ax7.scatter(np.arange(upper_lower_leg_angle_r_filtered.shape[0]),upper_lower_leg_angle_r_filtered, c='r',alpha=0.5)
    plt.title('Angle between upper leg & lower leg(right)')
    plt.xlabel('Frames')
    # plt.ylim(0,180) 
    ax8=fig.add_subplot(4,2,8)
    ax8.scatter(np.arange(upper_lower_leg_angle_l_filtered.shape[0]),upper_lower_leg_angle_l_filtered, c='r',alpha=0.5)
    plt.title('Angle between upper leg & lower leg(left)')
    plt.xlabel('Frames')
    # plt.ylim(0,180)
    plt.show()
